#### Importing dependencies

In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
import libpysal as ps
from mgwr.gwr import GWR, MGWR, GWRResults
from mgwr.sel_bw import Sel_BW
from scipy import stats
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


scaler = StandardScaler()



#### Initialize and view some rows

In [11]:
georgia_data = pd.read_csv(ps.examples.get_path('GData_utm.csv'))
georgia_shp = gp.read_file(ps.examples.get_path('G_utm.shp'))

g_y = georgia_data['PctBach'].values.reshape((-1,1))
# g_X = georgia_data[['PctFB', 'PctRural', 'PctBlack']].values
g_X = georgia_data[['PctEld']].values

u = georgia_data['X']
v = georgia_data['Y']

g_coords = list(zip(u,v))

georgia_data.head(3)

,AreaKey,Latitude,Longitud,TotPop90,PctRural,PctBach,PctEld,PctFB,PctPov,PctBlack,ID,X,Y
0,13001,31.75339,-82.28558,15744,75.6,8.2,11.43,0.64,19.9,20.76,133,941396.6,3521764.0
1,13003,31.29486,-82.87474,6213,100.0,6.4,11.77,1.58,26.0,26.86,158,895553.0,3471916.0
2,13005,31.55678,-82.45115,9566,61.7,6.6,11.11,0.27,24.1,15.42,146,930946.4,3502787.0


In [12]:
g_X.shape

(159, 1)

#### LWCC

In [19]:
gwr_selector = Sel_BW(g_coords, g_y, g_X)
gwr_bw = gwr_selector.search()
print(gwr_bw)
gwr_results = GWR(g_coords, g_y, g_X, gwr_bw, constant=False, lwcc=True).fit()
gwr_results.summary()

50.0
Model type                                                         Gaussian
Number of observations:                                                 159
Number of covariates:                                                     1

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                           8498.739
Log-likelihood:                                                    -541.923
AIC:                                                               1085.847
AICc:                                                              1087.924
BIC:                                                               7697.852
R2:                                                                  -0.657
Adj. R2:                                                             -0.657

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- -

### Check

In [20]:
corr = []

for i in range(len(g_X)):
    wi = gwr_results.model._build_wi(i, gwr_bw).reshape(-1,1)
    xw = g_X*wi
    yw = g_y*wi
    corr_coef, _ = stats.pearsonr(xw.flatten(), yw.flatten())
    corr.append(corr_coef)

np.array(corr).mean()

0.8139719277315267

### Correlation coefficient for each location

In [21]:
wi_OLS =  gwr_results.model._build_wi(2, gwr_bw).reshape(-1,1)

x = g_X
y= g_y

xi = x * wi_OLS
yi = y * wi_OLS

corr_coef, _ = stats.pearsonr(xi.flatten(), yi.flatten())
print(corr_coef)

0.9370513912157044


In [22]:
gwr_results.params[:10]

array([[0.91980963],
       [0.92363256],
       [0.93705139],
       [0.87975878],
       [0.87873139],
       [0.7344805 ],
       [0.71209177],
       [0.69459503],
       [0.93908483],
       [0.90555496]])

In [23]:
# Global correlation coefficient 

corr_coef, _ = stats.pearsonr(g_X.flatten(), g_y.flatten())
corr_coef

-0.4584957175756039

***

****

Rough area

In [17]:
# wi =  gwr_results.model._build_wi(0, gwr_bw).reshape(-1,1)
# # wi = np.sqrt(wi)

# xcc = g_X
# xcc_mean = (xcc).mean(axis=0)

# ycc = g_y
# ycc_mean = (ycc).mean(axis=0)

# numerator = (((xcc - xcc_mean) * (ycc - ycc_mean))*wi)
# denom = (np.sqrt((xcc - xcc_mean)**2)) * (np.sqrt((ycc - ycc_mean)**2))

# r = (numerator/denom)
# print(r)
# r.mean(axis=0)

In [18]:
# import math

# wi =  (gwr_results.model._build_wi(0, gwr_bw).reshape(-1,1))

# X = g_X
# Y = g_y

# Xmean = (sum(X*wi)/len(X))
# Ymean = (sum(Y*wi)/len(Y))

# x_diff = [var-Xmean for var in X]
# y_diff = [var-Ymean for var in Y]

# xy =([a*b for a,b in list(zip(x_diff,y_diff))])
# sum_xy = sum(xy)

# x_square = [a*a for a in x_diff]
# y_square = [b*b for b in y_diff]

# sum_x_square = sum(x_square)
# sum_y_square = sum(y_square)

# sum_x_square_sum_y_square = sum_x_square*sum_y_square
# sqrt_sum_x_square_sum_y_square = math.sqrt(sum_x_square_sum_y_square)

# r = sum_xy*wi/sqrt_sum_x_square_sum_y_square

# # print(wi*sum_xy)
# print(r)
